<a href="https://colab.research.google.com/github/drwbkr1/Grad504-Hierarchical-Cluster-Project/blob/main/GRAD504_HC_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

plants= ("/content/drive/MyDrive/Colab Notebooks/datasets/plants.data")

'''After discovering the correct encoding, I noticed the loader didn't like the
different number of datapoints in each line. So now I'm one-hot encoding the
dataset right out of the box'''

with open(plants, encoding='ISO-8859-1') as file:
  for line in file:
    items= line.strip().split() #turn each line into a list
    plants= items[0]            #reorganize the list into respective variables
    states= items[1:]
